In [1]:
#rendszer inicializálása

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
from bejelentkezo_v1 import bejelentkezo
from targyfelvevo_v1 import targyfelvevo

In [ ]:
neptun_code = 'EF764M'
password = '19NEPtun96'
maximum_time_waiting = 10
targy_lista = [
    {"name": "A mindennapi élet története 1500-2000-ig", "code": "E"},
    {"name": "Politikai gazdaságtan", "code": "G1"}
]

browser = bejelentkezo(neptun_code, password, maximum_time_waiting)

targyfelvevo(targy_lista, browser, maximum_time_waiting)

In [5]:
#chrome megnyitása
#neptun oldalának lekérése

browser = webdriver.Chrome()
url = 'https://neptun3r.web.uni-corvinus.hu/hallgatoi_1/login.aspx'
browser.get(url)

In [6]:
neptun_code = 'EF764M'
password = '19NEPtun96'
targy_neve = 'Statisztika I.'
szeminarium_kodja = 'G4'
maximum_time_waiting = 10

In [7]:
#vár, amíg a felhasználónév mező nem töltődik be

el = WebDriverWait(browser, maximum_time_waiting).until(lambda browser: browser.find_element_by_name("user"))

In [8]:
#beírja a megadott neptun kódot

browser.find_element_by_name('user').send_keys(neptun_code)

In [9]:
#beírja a megadott jelszót

browser.find_element_by_name('pwd').send_keys(password)

In [10]:
#megnyomja a belépés gombot

browser.find_element_by_name('btnSubmit').click()

In [11]:
#vár amíg a fenti menüsáv nem töltődik be (köztük a Tárgyak gomb)

el = WebDriverWait(browser, maximum_time_waiting).until(lambda browser: browser.find_element_by_id("mb1"))

In [12]:
# a Tárgyak gombra kattint, utána pedig azonnal a Tárgyfelvételre a lenyíló menüből

targyak_menu_gomb =browser.find_element_by_xpath("//ul[@id='mb1']/li[3]").click()
targyfelvetel_gomb = browser.find_element_by_xpath("//li[@id='mb1_Targyak']/ul/li[3]").click()

#Kiszámolja az adott félévet

currentMonth = datetime.now().month
currentYear = datetime.now().year
semester = 2

if 3 < currentMonth < 11:
    semester = 1
    currentYear += 1
elif currentMonth == 11 or 12:
    currentYear += 1

neptun_date = str(currentYear - 1) + '/' + str(currentYear)[-2:] + '/' + str(semester)
#70619 = 2019/20/2

#ez akkor működik jól, ha a szerver számokkal jelöli a szemesztereket (70619 = 2019/20/2)
neptun_date_option = browser.find_element_by_xpath("//table[@id = 'upFilter_bodytable']/tbody/tr[1]/td/select/option[@value = 70619]")
neptun_date_option.click()

In [13]:
#megvárja, amíg az adatmazők meg nem jelennek (maga a tábla, ami tartalmazza őket)

el = WebDriverWait(browser, maximum_time_waiting).until(lambda browser: browser.find_element_by_id("upFilter_bodytable"))

In [14]:
#ez akkor működik jól, ha mindig az első opció a legfrissebb szemeszter / ami szerintem mindig igaz
#kiválasztja a legfelső (legfrissebb) félévet

neptun_date_option = browser.find_element_by_xpath("//table[@id = 'upFilter_bodytable']/tbody/tr[1]/td/select/option[1]")
neptun_date_option.click()

In [15]:
browser.implicitly_wait(3)

In [16]:
#a tárgytípusok között bepipálja a mintatanterv tárgyai opciót (az "egyéb szabadon felvehető tárgyak" helyett)

mintatanterv_targyak_gomb = browser.find_element_by_xpath("//table[@id = 'upFilter_rbtnSubjectType']/tbody/tr[1]/td/input")
mintatanterv_targyak_gomb.click()

In [17]:
browser.implicitly_wait(3)

In [18]:
#kiválasztja hogy minden tárgy között keressen a kereső (mintatantervek = minden)

syllabus_option = browser.find_element_by_xpath("//table[@id = 'upFilter_bodytable']/tbody/tr[4]/td/select/option[@value = 'All']")
syllabus_option.click()

In [19]:
browser.implicitly_wait(5)

In [3]:
#kiválasztja, hogy minden tárgycsoport között keressen (tárgycsoportok = minden)

subject_classes_option = browser.find_element_by_xpath("//table[@id = 'upFilter_bodytable']/tbody/tr[5]/td[4]/select/option[@value = 'All']")
subject_classes_option.click()

In [87]:
browser.implicitly_wait(3)

In [24]:
#beállítja, hogy minden nyelvű kurzus kereshető legyen

subject_classes_option = browser.find_element_by_xpath("//table[@id = 'upFilter_bodytable']/tbody/tr[6]/td[4]/select/option[@value = 0]")
subject_classes_option.click()

In [89]:
browser.implicitly_wait(3)

In [90]:
#megkeresei a tárgynév mezőt, kitörli a tartalmát, belekattint és beírja a megadott tárgynevet, majd nyom egy entert.

subject_classes_option = browser.find_element_by_xpath("//table[@id = 'upFilter_bodytable']/tbody/tr[5]/td[2]/input")
subject_classes_option.clear()
subject_classes_option.click()
subject_classes_option.send_keys(targy_neve + Keys.RETURN)

In [91]:
#vár, amíg a tárgyakat kategorizáló felső sáv (tárgyneve, tárgy kódja (...) meg nem jelenik

el = WebDriverWait(browser, maximum_time_waiting).until(lambda browser: browser.find_element_by_id("h_addsubjects_gridSubjects_headerrow"))

#rákattint a legfelső találatra
found_seminars = browser.find_element_by_xpath("//table[@id = 'h_addsubjects_gridSubjects_bodytable']/tbody/tr[1]/td[2]/span").click()

In [92]:
for i in range(1,3):
    try:
        specific_seminar = browser.find_element_by_xpath("//table[@id = 'h_addsubjects_gridSubjects_bodytable']/tbody/tr[%i]/td[2]/span" %(i)).text
        if specific_seminar == targy_neve:
            browser.find_element_by_xpath("//table[@id = 'h_addsubjects_gridSubjects_bodytable']/tbody/tr[%i]/td[2]/span" %(i)).click()
            break
    except:
        print('Nem található ez a tárgynév!')


In [93]:
#megvárja, amíg a szemináriumokat tartalmazó tábla nem töltődik be.

el = WebDriverWait(browser, maximum_time_waiting).until(lambda browser: browser.find_element_by_id("Addsubject_course1_gridCourses_bodytable"))

specific_seminar = browser.find_element_by_xpath("//table[@id = 'Addsubject_course1_gridCourses_bodytable']/tbody/tr[2]/td[13]/input").click()

from fing2 import Fing

Fing()

f1 = lambda x, y : x + 2 * y

def f2(x, y):
    return x + 2 * y

type(lambda x: x)

In [ ]:
szeminarium_kodja = 'G4'
                                                 
for i in range(1,5):
    exist = True
    try:
        specific_seminar = browser.find_element_by_xpath("//table[@id = 'Addsubject_course1_gridCourses_bodytable']/tbody/tr[%i]/td[2]/span" %(i)).text
        if specific_seminar == szeminarium_kodja:
            specific_seminar = browser.find_element_by_xpath("//table[@id = 'Addsubject_course1_gridCourses_bodytable']/tbody/tr[i]/td[13]/input").click()
    except:
        print('Nem található ilyen kódú szeminárium!')

In [94]:
for i in range(1,40):
    exist = True
    try:
        found_seminar = browser.find_element_by_xpath("//table[@id = 'Addsubject_course1_gridCourses_bodytable']/tbody/tr[%i]/td[2]/span" %(i))
    except:
        exist = False
    if exist:
        if found_seminar.text == szeminarium_kodja:
            assert exist
            seminar_pipa = browser.find_element_by_xpath("//table[@id = 'Addsubject_course1_gridCourses_bodytable']/tbody/tr[%i]/td[13]/input" % i)
            seminar_pipa.click()

In [95]:
#rákattint a mentés gombra

save_button = browser.find_element_by_xpath("//table[@id = 'Addsubject_course1_gridCourses_tablebottom']/tbody/tr[1]/td[3]/table/tbody/tr/td/input").click()